## A/A testing of a mobile application

It is known that the split system is broken. It is required to check the statement about the breakage and find its causes, if the split-system is really broken

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as ss
from tqdm.auto import tqdm

### Defining Functions

In [2]:
# Function to print all information about DataFrame
def review_dataframe(df):
    print(" DATA INFO ".center(125,'-'))
    print(df.info())
    
    print(" SHAPE OF DATASET ".center(125,'-'))
    print('Rows:{}'.format(df.shape[0]))
    print('Columns:{}'.format(df.shape[1]))
    
    print(" DATA TYPES ".center(125,'-'))
    print(df.dtypes)
    
    print(" STATISTICS OF DATA ".center(125,'-'))
    print(df.describe(include="all"))
    
    print(" MISSING VALUES ".center(125,'-'))
    print(df.isnull().sum()[df.isnull().sum()>0].sort_values(ascending = False))
    
    print(" DUPLICATED VALUES ".center(125,'-'))
    print(df.duplicated().sum())

In [3]:
# Function for FRP metric
def get_fpr_metric(df_x, 
                   df_y, 
                   metric_column,
                   number_of_simulations,
                   percent,
                   boundary_observations,
                   estimator, #statistic estimator
                   *args,
                   **kwargs
                  ):
    
    statistic_results = {
                         'aa':{
                               'pvalue':[],
                               'mean_x': [],
                               'mean_y': []
                              }, 
                         'fpr':{
                                'fpr_95': 0
                               }
                        }
    
    for simulation in range(number_of_simulations):
        x = df_x[metric_column].sample(int(min(boundary_observations, len(df_x) * percent)), replace=True).values
        y = df_y[metric_column].sample(int(min(boundary_observations, len(df_y) * percent)), replace=True).values
    
        # choose statistic estimator:
        # t test
        if estimator == 'ttest':
            stat, pvalue = ss.ttest_ind(x, y, *args, **kwargs)
        # z test proportion
        elif estimator == 'prop':
            counts = np.array([sum(x), sum(y)])
            nobs = np.array([len(x), len(y)])
            stat, pvalue = proportions_ztest(counts, nobs, *args, **kwargs)
        else:
            raise 'Error: unknown estimator'
            
        statistic_results['aa']['pvalue'].append(pvalue)
        statistic_results['aa']['mean_x'].append(np.mean(x))
        statistic_results['aa']['mean_y'].append(np.mean(y))
        
    statistic_results['fpr']['fpr_95'] = float(sum(np.array(statistic_results['aa']['pvalue']) <= 0.05) / number_of_simulations)
    
    return statistic_results  

In [4]:
def get_fpr_report(df,
                   metric_column,
                   variant_column,
                   group_column,
                   number_of_simulations,
                   percent,
                   boundary_observations,
                   estimator, #statistic estimator
                   *args,
                   **kwargs
                  ):
    list_fpr = []
    list_group = list(pd.unique(df[group_column]))
    
    for version in range(len(list_group)):
        df_x = df[ (df[variant_column] == 0) & (df[group_column] == list_group[version]) ]
        df_y = df[ (df[variant_column] == 1) & (df[group_column] == list_group[version]) ]
        
        cr_x = sum(df_x[metric]) / len(df_x)
        cr_y = sum(df_y[metric]) / len(df_y)
        
        fpr = {}
        fpr = get_fpr_metric(
            df_x = df_x,
            df_y = df_y,
            metric_column = metric,
            number_of_simulations = number_of_simulations,
            percent = percent,
            boundary_observations = boundary_observations,
            estimator = estimator, *args, **kwargs
        )
        
        is_fpr = (fpr['fpr']['fpr_95']<=0.05)
        list_fpr.append([list_group[version], cr_x, cr_y, fpr['fpr']['fpr_95'], is_fpr])
    
    report = pd.DataFrame.from_records(list_fpr, columns=['group', 'cr_x', 'cr_y', 'fpr_95', 'is_fpr'])
    
    
    return report

### Main Part

In [5]:
df_main = pd.read_csv('data/for_aa.csv', decimal=',', sep=';')
df_main.head()

,Unnamed: 0,uid,experimentVariant,version,purchase
0,1,c4ca4238a0b923820dcc509a6f75849b,1,v2.8.0,0
1,2,c81e728d9d4c2f636f067f89cc14862c,0,v2.9.0,0
2,3,eccbc87e4b5ce2fe28308fd9f2a7baf3,1,v2.9.0,0
3,4,a87ff679a2f3e71d9181a67b7542122c,1,v2.8.0,0
4,5,e4da3b7fbbce2345d7772b0674a318d5,1,v2.8.0,0


In [6]:
review_dataframe(df_main)

--------------------------------------------------------- DATA INFO ---------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127018 entries, 0 to 127017
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Unnamed: 0         127018 non-null  int64 
 1   uid                127018 non-null  object
 2   experimentVariant  127018 non-null  int64 
 3   version            127018 non-null  object
 4   purchase           127018 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 4.8+ MB
None
------------------------------------------------------ SHAPE OF DATASET -----------------------------------------------------
Rows:127018
Columns:5
--------------------------------------------------------- DATA TYPES --------------------------------------------------------
Unnamed: 0            int64
uid                  object
experimentVariant     int64
version        

Description:

    experimentVariant – type of experiment
    version – mobile app versioan
    purchase – fact of purchase

In [7]:
n_sim = 1000
n_s_perc = 0.9
n_s_min = 1000
metric = 'purchase'
variant = 'experimentVariant'
group = 'version'

res = get_fpr_report(
    df = df_main, 
    metric_column=metric, 
    variant_column=variant, 
    group_column=group, 
    number_of_simulations=n_sim, 
    percent=n_s_perc, 
    boundary_observations=n_s_min, 
    estimator='ttest'
)
res

,group,cr_x,cr_y,fpr_95,is_fpr
0,v2.8.0,0.000993,0.045606,1.000,False
1,v2.9.0,0.074658,0.071304,0.052,False
2,v3.7.4.0,0.059943,0.063018,0.065,False
3,v3.8.0.0,0.057604,0.062848,0.076,False
